In [47]:
import tensorflow as tf
import numpy as np

import os
import json
from pinn import PINNModel, model1, model2
from data_gen import DataGenerator, real_u1, get_koefs

In [48]:
# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [ ]:
area_grid_size = (50, 50)
border_grid_size = (50, 50)
x_limits = (0, 2)
y_limits = (0, 2)

EPOCHS = 5000

# Define the learning rate schedule
boundaries = [5000, 15000, 50000]
values = [1e-3, 1e-4, 1e-5, 1e-6]

lr_sched = tf.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)

model = PINNModel(
    model1(), optm=tf.keras.optimizers.Adam(learning_rate=lr_sched))

dataGen = DataGenerator(x_limits, y_limits, model.predict, real_u1)

In [ ]:
from accuracy import AccuracyCalc
from plotting import NNPlots

koefs = get_koefs()

session = "session7"

initial_weights_path = f"../models/s={session}.initial.weights.h5"

data_set = dataGen.inner_pairs(area_grid_size), dataGen.border_pairs(border_grid_size)

data = open('./koef.json', 'r')

reset = False
if not session in data or reset == True:
    model.save(initial_weights_path)
    # data[f'session'] = koefs
    for koef in koefs:
        path = f"../models/s={session}.model{koef}.weights.h5"
        model = PINNModel(
    model1(), optm=tf.keras.optimizers.Adam(learning_rate=lr_sched))
        model.load(initial_weights_path)
        model.fit(koef, *data_set, EPOCHS, EPRINT=EPOCHS)
        model.save(path)
    # with open('./koef.json', 'w') as file:
    #     json.dump(data, file, indent=4)
# model.load(f"../models/s={session}.model{3}.weights.h5")
# model.load(initial_weights_path)
# model
# data_set = dataGen.inner_pairs(area_grid_size), dataGen.border_pairs(border_grid_size)
# model.fit(3, *data_set, EPOCHS=5000, EPRINT=500)
# acc = AccuracyCalc(dataGen, (70, 70))
# print(f'mse_loss: {acc.mse()}')

In [ ]:
# model.load(f"../models/s={session}.model{3}.weights.h5")
model.load(initial_weights_path)
model = PINNModel(
    model1(), optm=tf.keras.optimizers.Adam(learning_rate=lr_sched))
dataGen = DataGenerator(x_limits, y_limits, model.predict, real_u1)
model.load(initial_weights_path)
data_set = dataGen.inner_pairs(area_grid_size), dataGen.border_pairs(border_grid_size)
model.fit(1, *data_set, EPOCHS=500, EPRINT=1000)
acc = AccuracyCalc(dataGen, (70, 70))
print(f'mse_loss: {acc.mse()}')

In [ ]:
plotter = NNPlots(dataGen, (70, 70))
plotter.plot2d_contour()